# Introducing Q-Learning

## What is Q-Learning?

Q-Learning, **eylem-değer fonksiyonunu eğitmek için bir TD yaklaşımı kullanan politika dışı değer tabanlı bir yöntemdir:**

- *Politika dışı*: bu ünitenin sonunda bundan bahsedeceğiz 
- *Değer tabanlı yöntem*: bize **her bir durumun veya her bir durum-eylem çiftinin değerini söyleyecek bir değer veya eylem-değer fonksiyonunu eğiterek dolaylı olarak en uygun politikayı bulur.**
- *TD yaklaşımı:* **eylem-değer fonksiyonunu bölümün sonunda değil her adımda günceller.**

**Q-Learning, belirli bir durumda olmanın ve o durumda belirli bir eylemde bulunmanın değerini belirleyen bir **eylem-değer fonksiyonu** olan Q-fonksiyonumuzu** eğitmek için kullandığımız algoritmadır.

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-function.jpg)

Q, o eylemin o durumdaki "Quality" (değerinden) gelir.**

Değer ve ödül arasındaki farkı özetleyelim:

- Bir durumun ya da bir *durum-eylem çiftinin* değeri, ajanımızın bu durumdan (ya da durum-eylem çiftinden) başlayıp politikasına göre hareket etmesi halinde elde edeceği beklenen kümülatif ödüldür - *ödül*, bir durumda bir eylem gerçekleştirdikten sonra **çevreden aldığım** geribildirimdir.

Dahili olarak, Q-fonksiyonumuz **her hücrenin bir durum-eylem çifti değerine karşılık geldiği bir tablo olan bir Q-tablosu ile kodlanır.** Bu Q-tablosunu **Q-fonksiyonumuzun hafızası veya kopya kağıdı olarak düşünün.**

Bir labirent örneği üzerinden gidelim.

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Maze-1.jpg)

Q-tablosu başlatılmıştır. Bu nedenle tüm değerler = 0'dır. Bu tablo, her durum ve eylem için karşılık gelen durum-eylem değerlerini içerir. Bu basit örnek için, durum yalnızca farenin konumu ile tanımlanır. Bu nedenle, Q-tablomuzda 2*3 satırımız var, farenin her olası konumu için bir satır. Daha karmaşık senaryolarda, durum aktörün konumundan daha fazla bilgi içerebilir.

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Maze-2.jpg)

Burada başlangıç durumunun ve yukarı çıkmanın durum-eylem değerinin 0 olduğunu görüyoruz:

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Maze-3.jpg)

Yani: Q-fonksiyonu her durum-eylem çiftinin değerini içeren bir Q-tablosu kullanır. Bir durum ve eylem verildiğinde, Q-fonksiyonumuz değeri çıktılamak için Q-tablosunun içinde arama yapacaktır.

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-function-2.jpg)

Özetleyecek olursak, *Q-Öğrenme* **aşağıdaki RL algoritmasıdır:**

- Dahili olarak tüm durum-eylem çifti değerlerini içeren bir **Q-tablosu** olan bir *Q-fonksiyonu* (bir **eylem-değer fonksiyonu**) eğitir.**
- Bir durum ve eylem verildiğinde, Q-fonksiyonumuz **karşılık gelen değeri bulmak için Q-tablosunda arama yapar.** 
- Eğitim tamamlandığında, **optimal bir Q-fonksiyonuna sahip oluruz, bu da optimal Q-tablosuna sahip olduğumuz anlamına gelir.** 
- Ve eğer **optimal bir Q-fonksiyonuna** sahipsek, **optimal bir politikaya** sahip oluruz, çünkü **her durumda yapılacak en iyi eylemi biliriz.**

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/link-value-policy.jpg)

Başlangıçta, Q-tablomuz her durum-eylem çifti için keyfi değerler verdiğinden işe yaramaz (çoğu zaman Q-tablosunu 0 olarak başlatırız). Temsilci çevreyi keşfettikçe ve biz Q-tablosunu güncelledikçe, bu bize optimal politikaya giderek daha iyi bir yaklaşım sağlayacaktır.

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-learning-1.jpg)

Artık Q-Learning, Q-fonksiyonları ve Q-tablolarının ne olduğunu anladığımıza göre, Q-Learning algoritmasının derinliklerine inelim.

## The Q-Learning algorithm

Bu Q-Learning sözde kodudur; uygulamadan önce her bir parçayı inceleyelim ve basit bir örnekle nasıl çalıştığını görelim. Bu gözünüzü korkutmasın, göründüğünden daha basittir! Her adımın üzerinden geçeceğiz.

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-learning-2.jpg)

**Step 1: We initialize the Q-table**

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-learning-3.jpg)
Her durum-eylem çifti için Q-tablosunu başlatmamız gerekir. Çoğu zaman 0 değeriyle başlatırız.

**Step 2: Choose an action using the epsilon-greedy strategy**

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-learning-4.jpg)

Epsilon-greedy stratejisi, keşif/sömürü dengesini ele alan bir politikadır.

Buradaki fikir, ɛ = 1.0 başlangıç değeri ile:

- 1 - ɛ* olasılığı ile: **sömürü** yaparız (diğer bir deyişle ajanımız en yüksek durum-eylem çifti değerine sahip eylemi seçer) 
- ɛ olasılığı ile: **keşif** yaparız (rastgele eylem deneriz).

Eğitimin başında, ɛ çok yüksek olduğu için **keşif yapma olasılığı çok büyük olacaktır, bu nedenle çoğu zaman keşif yapacağız.** Ancak eğitim devam ettikçe ve sonuç olarak **Q-tablomuz tahminlerinde gittikçe daha iyi hale geldikçe, epsilon değerini** giderek azaltıyoruz çünkü daha az keşfe ve daha fazla sömürüye ihtiyacımız olacak.

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-learning-5.jpg)

**Step 3: Perform action At, get reward Rt+1 and next state St+1**

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-learning-6.jpg)

**Step 4: Update Q(St, At)**

TD Öğrenmede, politikamızı veya değer fonksiyonumuzu (seçtiğimiz RL yöntemine bağlı olarak) **etkileşimin bir adımından sonra** güncellediğimizi unutmayın.

TD hedefimizi üretmek için, **anlık ödül $R_{t+1}$ artı bir sonraki durumun iskonto edilmiş değerini** kullandık, bir sonraki durumda mevcut Q-fonksiyonunu maksimize eden eylemi bularak hesapladık. (Biz buna bootstrap diyoruz).

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-learning-7.jpg)

Bu nedenle, $Q(S_t, A_t)$ **güncelleme formülü şu şekildedir:**

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-learning-8.jpg)

Bu, $Q(S_t, A_t)$ değerimizi güncellemek anlamına gelir:

- $S_t, A_t, R_{t+1}, S_{t+1}$'e ihtiyacımız var. 
- Belirli bir durum-eylem çiftindeki Q değerimizi güncellemek için TD hedefini kullanırız.

TD hedefini nasıl oluştururuz? 
1. $A_t$ eylemini gerçekleştirdikten sonra $R_{t+1}$ ödülünü elde ederiz. 
2. Bir sonraki durum için **en iyi durum-eylem çifti değerini** elde etmek amacıyla, bir sonraki en iyi eylemi seçmek için açgözlü bir politika kullanırız. Bunun bir epsilon-greedy politikası olmadığına dikkat edin, bu her zaman en yüksek durum-eylem değerine sahip eylemi alacaktır.

Daha sonra bu Q-değerinin güncellemesi tamamlandığında, yeni bir durumda başlarız ve eylemimizi **yine bir epsilon-greedy politikası kullanarak** seçeriz.

**Bu yüzden Q Learning'in politika dışı bir algoritma olduğunu söylüyoruz.**

## Off-policy vs On-policy [[off-vs-on]]

Aradaki fark çok ince:

- *Off-policy*: **eylem (çıkarım) ve güncelleme (eğitim) için farklı bir politika** kullanmak.

Örneğin, Q-Learning ile epsilon-greedy politikası ( eylem politikası), Q-değerimizi güncellemek için en iyi sonraki durum eylem değerini seçmek için kullanılan greedy politikasından (güncelleme politikası) farklıdır.**

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/off-on-1.jpg)

Eğitim bölümünde kullandığımız politikadan farklıdır:

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/off-on-2.jpg)

- *On-policy:* *eylem ve güncelleme için aynı politika** kullanmak.

Örneğin, başka bir değer tabanlı algoritma olan Sarsa'da **epsilon-greedy politikası bir sonraki durum-eylem çiftini seçer, greedy bir politika değil, **.

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/off-on-3.jpg)

![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/off-on-4.jpg)